# Lab-7 : 긴 문장 가지고 RNN 기본형 학습하기

## https://github.com/pkmital/CADL 에서 가져옴

밑은 tensorflow 라이브러리를 import하고 있고, 이후 이 모듈을 tf라는 이름으로 사용하겠다는 뜻이다

In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
from six.moves import urllib

다음 명령문은 http://www.gutenberg.org/files/11/11-0.txt 링크의 텍스트를 추출하여 'alice.txt'라는 파일로 저장하고, f라는 파일변수

In [4]:
f, _ = urllib.request.urlretrieve('http://www.gutenberg.org/files/11/11-0.txt', 'alice.txt')

In [5]:
with open(f, 'r') as fp:
    txt = fp.read()

set(txt)는 txt내의 모든 글자의 종류를 추출한 집합 객체를 반환해준다. 이를 list함수에 의해 배열로 만든 뒤 vocab에 저장한다.
len(txt)를 통해 txt의 전체글자수를 알 수 있고, len(vocab)에 의해 글자의 종류수를 알 수 있다.

In [6]:
vocab = list(set(txt))
len(txt), len(vocab)

(163817, 87)

In [7]:
print(vocab)

['z', 'A', '0', 'a', ';', 'j', '!', 'U', 'Y', 'm', '/', 'X', 'R', 'e', '”', 'w', '[', '(', '-', 'x', '*', '1', 'v', '4', '9', 's', 'E', 'k', 'p', 'M', 'P', 'J', '\n', 'f', 'H', 'L', 'N', 'V', 'r', 'g', '’', 'l', 'i', 'F', '5', 'S', 'C', '\ufeff', 'Q', '‘', ' ', 'y', '@', '8', '%', 'I', '$', 'd', '?', 'T', '6', '“', 'K', 'h', 't', 'D', 'q', 'b', '_', ',', 'o', 'u', 'n', '.', '2', 'B', '#', 'W', ')', '3', '7', 'O', 'Z', ']', 'G', 'c', ':']


In [8]:
print(range(len(vocab)))

range(0, 87)


다음에는 각 문자를 특정 정수와 대응하는 사전을 만든다.
encoder는 문자 -> 정수에 대한 사전이고
decoder는 정수 -> 문자에 대한 사전이다

In [9]:
encoder = dict(zip(vocab, range(len(vocab))))

In [10]:
print(encoder)

{'z': 0, '[': 16, 'A': 1, '0': 2, 'a': 3, '’': 40, 'j': 5, '!': 6, 'U': 7, 'Y': 8, 'm': 9, 'g': 39, '‘': 49, '(': 17, ' ': 50, '/': 10, '\ufeff': 47, 'X': 11, 'y': 51, 'R': 12, 'e': 13, '@': 52, '8': 53, '%': 54, 'I': 55, '$': 56, 'd': 57, ';': 4, '?': 58, 'T': 59, '6': 60, '“': 61, 'K': 62, '-': 18, 'x': 19, 'F': 43, '*': 20, 'h': 63, 'D': 65, 'i': 42, '1': 21, 'Q': 48, 'v': 22, '4': 23, '”': 14, '9': 24, 'q': 66, 's': 25, 'b': 67, '_': 68, 'E': 26, 'o': 70, 'u': 71, 'M': 29, 'n': 72, ',': 69, 'P': 30, '.': 73, 'J': 31, '2': 74, '\n': 32, 'B': 75, 'k': 27, '#': 76, 't': 64, 'p': 28, 'f': 33, 'H': 34, 'w': 15, 'W': 77, 'L': 35, 'N': 36, ')': 78, 'V': 37, 'r': 38, '3': 79, 'l': 41, '7': 80, 'O': 81, 'Z': 82, ']': 83, 'G': 84, 'c': 85, '5': 44, ':': 86, 'S': 45, 'C': 46}


In [11]:
decoder = dict(zip(range(len(vocab)), vocab))

In [12]:
print(decoder)

{0: 'z', 1: 'A', 2: '0', 3: 'a', 4: ';', 5: 'j', 6: '!', 7: 'U', 8: 'Y', 9: 'm', 10: '/', 11: 'X', 12: 'R', 13: 'e', 14: '”', 15: 'w', 16: '[', 17: '(', 18: '-', 19: 'x', 20: '*', 21: '1', 22: 'v', 23: '4', 24: '9', 25: 's', 26: 'E', 27: 'k', 28: 'p', 29: 'M', 30: 'P', 31: 'J', 32: '\n', 33: 'f', 34: 'H', 35: 'L', 36: 'N', 37: 'V', 38: 'r', 39: 'g', 40: '’', 41: 'l', 42: 'i', 43: 'F', 44: '5', 45: 'S', 46: 'C', 47: '\ufeff', 48: 'Q', 49: '‘', 50: ' ', 51: 'y', 52: '@', 53: '8', 54: '%', 55: 'I', 56: '$', 57: 'd', 58: '?', 59: 'T', 60: '6', 61: '“', 62: 'K', 63: 'h', 64: 't', 65: 'D', 66: 'q', 67: 'b', 68: '_', 69: ',', 70: 'o', 71: 'u', 72: 'n', 73: '.', 74: '2', 75: 'B', 76: '#', 77: 'W', 78: ')', 79: '3', 80: '7', 81: 'O', 82: 'Z', 83: ']', 84: 'G', 85: 'c', 86: ':'}


모델의 파라미터를 정한다.

In [13]:
# batch에 들어가는 sequence의 수
batch_size = 100

# 시퀀스안에 들어가는 문자의 갯수
sequence_length = 100

# LSTM 레이어 안의 cell의  갯수
n_cells = 256

# LSTM레이어의 갯수
n_layers = 2

# one-hot encoding하려고 할 때의 들어가는 수의 갯수
n_chars = len(vocab)

In [14]:
#학습데이터를 넣을 공간 준비
X = tf.placeholder(tf.int32, [None, sequence_length], name='X')

# 참값인 문장이 들어갈 공간 준비
Y = tf.placeholder(tf.int32, [None, sequence_length], name='Y')

In [15]:
# we first create a variable to take us from our one-hot representation to our LSTM cells
embedding = tf.get_variable("embedding", [n_chars, n_cells])

In [16]:
print(embedding.get_shape())

(87, 256)


In [17]:
# And then use tensorflow's embedding lookup to look up the ids in X
Xs = tf.nn.embedding_lookup(embedding, X)

In [18]:
print(Xs.get_shape())

(?, 100, 256)


In [19]:
# The resulting lookups are concatenated into a dense tensor
print(Xs.get_shape().as_list())

[None, 100, 256]


In [20]:
# Let's create a name scope for the operations to clean things up in our graph
with tf.name_scope('reslice'):
    Xs = [tf.squeeze(seq, [1])
          for seq in tf.split(1, sequence_length, Xs)]

In [21]:
cells = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_cells, state_is_tuple=True)

In [22]:
initial_state = cells.zero_state(tf.shape(X)[0], tf.float32)

In [23]:
# this will return us a list of outputs of every element in our sequence.
# Each output is `batch_size` x `n_cells` of output.
# It will also return the state as a tuple of the n_cells's memory and
# their output to connect to the time we use the recurrent layer.
outputs, state = tf.nn.rnn(cells, Xs, initial_state=initial_state)

# We'll now stack all our outputs for every cell
outputs_flat = tf.reshape(tf.concat(1, outputs), [-1, n_cells])

In [24]:
with tf.variable_scope('prediction'):
    W = tf.get_variable(
        "W",
        shape=[n_cells, n_chars],
        initializer=tf.random_normal_initializer(stddev=0.1))
    b = tf.get_variable(
        "b",
        shape=[n_chars],
        initializer=tf.random_normal_initializer(stddev=0.1))

    # Find the output prediction of every single character in our minibatch
    # we denote the pre-activation prediction, logits.
    logits = tf.matmul(outputs_flat, W) + b

    # We get the probabilistic version by calculating the softmax of this
    probs = tf.nn.softmax(logits)

    # And then we can find the index of maximum probability
    Y_pred = tf.argmax(probs, 1)

Loss Function을 정의한다

In [25]:
with tf.variable_scope('loss'):
    # Compute mean cross entropy loss for each output.
    Y_true_flat = tf.reshape(tf.concat(1, Y), [-1])
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, Y_true_flat)
    mean_loss = tf.reduce_mean(loss)

Clipping Gradient (그레디어값이 너무 커지지 않게 함)

In [26]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    gradients = []
    clip = tf.constant(5.0, name="clip")
    for grad, var in optimizer.compute_gradients(mean_loss):
        gradients.append((tf.clip_by_value(grad, -clip, clip), var))
    updates = optimizer.apply_gradients(gradients)

Training

In [27]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

cursor = 0
it_i = 0
while True:
    Xs, Ys = [], []
    for batch_i in range(batch_size):
        if (cursor + sequence_length) >= len(txt) - sequence_length - 1:
            cursor = 0
        Xs.append([encoder[ch]
                   for ch in txt[cursor:cursor + sequence_length]])
        Ys.append([encoder[ch]
                   for ch in txt[cursor + 1: cursor + sequence_length + 1]])
       
        cursor = (cursor + sequence_length)
    Xs = np.array(Xs).astype(np.int32)
    Ys = np.array(Ys).astype(np.int32)

    loss_val, _ = sess.run([mean_loss, updates],
                           feed_dict={X: Xs, Y: Ys})
    print(it_i, loss_val)

    if it_i % 300 == 0:
        p = sess.run([Y_pred], feed_dict={X: Xs})[0]
        preds = [decoder[p_i] for p_i in p]
        print("".join(preds).split('\n'))

    it_i += 1

0 4.47436
['k8kkk88g8ksssrssskkrtkkrrrrrr8rrrrrrrrrrsrrrtrrrrrrrrrrrrrk(((((((((ttrtggrrrrrrkkkkr(((kkkkrrrrrrr(kkkkkkkkkkkrrrrrrr(rrrkkrktrt(rrrr(rr((rrrrrrrtttrrrrsrrssrrrrrrrrrrrrrrrr(tt(r(r(rtkkk((rtk(rr(sss(r(r8kktt888rssssssrrrrrrrrrrrrssrtkkkrrrrrkrrrrrssssrsssstksrsrrssssrrrrrrrr(rrrrrrrrrrttkttttkkkrrr8(88(((rRrrrrrsrRrttttttLssgrsskkrrrrrr8rrrrrrrrrrsrrrtrrrrrrrrrrrrrrrrrrkrr(((((((ttrtgr0\ufeffrrrrrrrrrk888888((kk(((((((((0k((k‘r((8rr((s((((rs((((rrrrrrrr(((((((((((rrrrrrrttsrtrrrrrrrggL(rrrrr8r8((RHrrRRR8Rrrrrrrrarsgassrkrrrrrrrtkk((((00kk88888kkkkrkrkkrkk((((kkL((k(((((((k((88LL(LL(888((((8(((((((kk8(8888888kk(((@@@00000080LLkkkk88((((8(((((((kk((((8k8(LL8((((((((ykktgggg((kkLL(k(kL(kkLL(LL(((((kkk\ufeff\ufeff\ufeffrgggL0LL888L88krr(0(((rrrr(((ssss(kkkgg8rsrrrr((((s8ssrsryrrttt8ssrsrrrrsssrrtkk((ssssrtstt(rr8(((rsrrkkkrrrrsrrrrrrrrrkkk(((((k(rkrrrrrrrttttkkkkrrrkkkkr(rrr((r(((rr(((((((((ss(srrrsstkkrrrrr((r888r((((r(rrrrrrtrsrsssr(rrr((((rrrrrsrtttrtrrrrrrrssrrrsrsssrrrrrr

KeyboardInterrupt: 